In [2]:
from pathlib import Path

import skimage
from skimage import io
from skimage.transform import resize
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from skimage.filters import laplace

%matplotlib inline

In [3]:
def laplace_multiplicand(fileID):
    GOES = io.imread(fileID);
    GOES0 = np.squeeze(GOES[:,:,0])
    height, width = GOES0.shape
    smallGOES0 = resize(GOES0, (round(height/2), round(width/2)))*255
    blurryGOES0 = resize(smallGOES0, (height, width))
    details = GOES0 - blurryGOES0
    laplace_edges = laplace(blurryGOES0)
    
    table = []
    multiplicand = 0
    while multiplicand <= 1:
        sharped_image = blurryGOES0+laplace_edges * multiplicand
        abs_error = abs(GOES0-sharped_image)
        mean = abs_error.mean()
        sqrt = np.sqrt(((details- laplace_edges * multiplicand)**2).mean())
        table.append({'Filename': fileID,'Multiplicand': multiplicand, 'Mean Error': mean, 'Square Root': sqrt})
        multiplicand = multiplicand + .1
    return pd.DataFrame(table)

In [4]:
laplace_multiplicand('GOES7.tif')

,Filename,Multiplicand,Mean Error,Square Root
0,GOES7.tif,0.0,1.324904,1.720790
1,GOES7.tif,0.1,1.296801,1.677800
2,GOES7.tif,0.2,1.273069,1.641438
3,GOES7.tif,0.3,1.254313,1.612153
4,GOES7.tif,0.4,1.240880,1.590336
5,GOES7.tif,0.5,1.232684,1.576297
6,GOES7.tif,0.6,1.229762,1.570245
7,GOES7.tif,0.7,1.232025,1.572272
8,GOES7.tif,0.8,1.239278,1.582347
9,GOES7.tif,0.9,1.251211,1.600318


In [8]:
all_dfs = pd.concat(laplace_multiplicand(file) for file in Path('Fires/TIFF_Files/CampFire/').iterdir())

FileNotFoundError: [Errno 2] No such file or directory: 'Fires/TIFF_Files/CampFire'